# Pruned and Unpruned NH-AMPCs 

## Imports

In [1]:
import os
import numpy as np
import torch

from itertools import product
from pprint import pprint
from tqdm.notebook import tqdm_notebook

In [2]:
from src.parameters import NH_AMPC_Param
from src.generate_trajs import generate_NH_AMPC_trajs

## Settings

In [3]:
RESULTS_DIR = os.path.abspath('Results')
MPC_DATASETS_DIR = os.path.join(RESULTS_DIR, 'MPC_data_gen')
NNS_DIR = os.path.join(RESULTS_DIR, 'Trained_Networks')
MPC_RESULTS_DIR = os.path.join(RESULTS_DIR, 'NH_AMPC_results_prun')

MAKE_TEST_RESULTS = False

if MAKE_TEST_RESULTS:
    MPC_RESULTS_DIR += '_Test'

NUM_SAMPLES = 30_000

NUM_NNS = 50
USE_CUDA = False

MPC_PARAM_DICT = {
    'T_sim': 5, # length of the closed-loop simulation (in seconds)
}

PRUNED_NH_AMPC_OPTIONS = [tup for tup in product(
    (8, ),                                      # N_MPC
    (17, ),                                     # N_NN -> if USE_BEGINING_OF_DATASET != 'begin, it has to be 70-max(N_MPCs)
    (70, ),                                     # N_DATASET
    (5, ),                                      # TRAIN_DATASET_VERSION
    (6, ),                                      # TEST_DATASET_VERSION
    ('fixed', ),                                # USE_BEGINING_OF_DATASET ('begin', 'fixed', '') if '' use DS_FEATURE of N_MPC
    (8, ),                                      # DS_FEATURE_IF_FIXED
    ('RTI_PCHPIPM_DISCRETE', ),                 # DATASET_NAME_ADD ('RTI_PCHPIPM_DISCRETE', 'RTI_PCHPIPM_DISCRETE_50ITER', 'RTI_PCHPIPM_ROBUST_DISCRETE')
    (48, ),                                     # HIDDEN_NEURONS
    (24, ),                           # END_HIDDEN_SIZES (left from pruning)
)]

PRUNED_NH_AMPC_OPTIONS.extend([tup for tup in product(
    (8, ),                                      # N_MPC
    (22, ),                                     # N_NN -> if USE_BEGINING_OF_DATASET != 'begin, it has to be 70-max(N_MPCs)
    (70, ),                                     # N_DATASET
    (5, ),                                      # TRAIN_DATASET_VERSION
    (6, ),                                      # TEST_DATASET_VERSION
    ('fixed', ),                                # USE_BEGINING_OF_DATASET ('begin', 'fixed', '') if '' use DS_FEATURE of N_MPC
    (8, ),                                      # DS_FEATURE_IF_FIXED
    ('RTI_PCHPIPM_DISCRETE', ),                 # DATASET_NAME_ADD ('RTI_PCHPIPM_DISCRETE', 'RTI_PCHPIPM_DISCRETE_50ITER', 'RTI_PCHPIPM_ROBUST_DISCRETE')
    (64, ),                                     # HIDDEN_NEURONS
    (24, 32, ),                           # END_HIDDEN_SIZES (left from pruning)
)])


UNPRUNED_NH_AMPC_OPTIONS = [tup for tup in product(
    (8, ),                                      # N_MPC
    (17, ),                                     # N_NN -> if USE_BEGINING_OF_DATASET != 'begin, it has to be 70-max(N_MPCs)
    (70, ),                                     # N_DATASET
    (5, ),                                      # TRAIN_DATASET_VERSION
    (6, ),                                      # TEST_DATASET_VERSION
    ('fixed', ),                                # USE_BEGINING_OF_DATASET ('begin', 'fixed', '') if '' use DS_FEATURE of N_MPC
    (8, ),                                      # DS_FEATURE_IF_FIXED
    ('RTI_PCHPIPM_DISCRETE', ),                 # DATASET_NAME_ADD ('RTI_PCHPIPM_DISCRETE', 'RTI_PCHPIPM_DISCRETE_50ITER', 'RTI_PCHPIPM_ROBUST_DISCRETE')
    (24, 48, ),                                     # HIDDEN_NEURONS
    (None, ),                           # END_HIDDEN_SIZES (left from pruning)
)]

UNPRUNED_NH_AMPC_OPTIONS.extend([tup for tup in product(
    (8, ),                                      # N_MPC
    (22, ),                                     # N_NN -> if USE_BEGINING_OF_DATASET != 'begin, it has to be 70-max(N_MPCs)
    (70, ),                                     # N_DATASET
    (5, ),                                      # TRAIN_DATASET_VERSION
    (6, ),                                      # TEST_DATASET_VERSION
    ('fixed', ),                                # USE_BEGINING_OF_DATASET ('begin', 'fixed', '') if '' use DS_FEATURE of N_MPC
    (8, ),                                      # DS_FEATURE_IF_FIXED
    ('RTI_PCHPIPM_DISCRETE', ),                 # DATASET_NAME_ADD ('RTI_PCHPIPM_DISCRETE', 'RTI_PCHPIPM_DISCRETE_50ITER', 'RTI_PCHPIPM_ROBUST_DISCRETE')
    (24, 32, 64, ),                                     # HIDDEN_NEURONS
    (None, ),                           # END_HIDDEN_SIZES (left from pruning)
)])


COMBINED_NH_AMPC_OPTIONS = [*UNPRUNED_NH_AMPC_OPTIONS, *PRUNED_NH_AMPC_OPTIONS]

In [4]:
# sort the options so they are performed from smallest networks to biggest networks.
# First the unpruned one then the pruned one.
COMBINED_NH_AMPC_OPTIONS = sorted(COMBINED_NH_AMPC_OPTIONS, key=lambda x: (x[8] if x[9] is None else x[9]))
pprint(COMBINED_NH_AMPC_OPTIONS)
print(NNS_DIR)
print(MPC_RESULTS_DIR)

[(8, 17, 70, 5, 6, 'fixed', 8, 'RTI_PCHPIPM_DISCRETE', 24, None),
 (8, 22, 70, 5, 6, 'fixed', 8, 'RTI_PCHPIPM_DISCRETE', 24, None),
 (8, 17, 70, 5, 6, 'fixed', 8, 'RTI_PCHPIPM_DISCRETE', 48, 24),
 (8, 22, 70, 5, 6, 'fixed', 8, 'RTI_PCHPIPM_DISCRETE', 64, 24),
 (8, 22, 70, 5, 6, 'fixed', 8, 'RTI_PCHPIPM_DISCRETE', 32, None),
 (8, 22, 70, 5, 6, 'fixed', 8, 'RTI_PCHPIPM_DISCRETE', 64, 32),
 (8, 17, 70, 5, 6, 'fixed', 8, 'RTI_PCHPIPM_DISCRETE', 48, None),
 (8, 22, 70, 5, 6, 'fixed', 8, 'RTI_PCHPIPM_DISCRETE', 64, None)]
/home/jovyan/work/acadosmpc/Results/Trained_Networks
/home/jovyan/work/acadosmpc/Results/NH_AMPC_results_prun


In [5]:
NH_AMPC_PARAMS = [
    NH_AMPC_Param(
        # Param
        N_MPC = N_MPC, 
        N = N_NN+N_MPC,

        # Dataset stuff
        N_DS = N_DSET, 
        TRAIN_V_DS = TRAIN_DATASET_VERSION, 
        TEST_V_DS = TEST_DATASET_VERSION, 
        DS_begin = USE_BEGIN,
        DS_samples = NUM_SAMPLES,
        DS_opts_name = DS_OPT_NAME,
        DS_feature = DS_FEATURES,

        # NN stuff
        V_NN = NN_VERSION,
        N_hidden = N_HIDDEN,
        N_hidden_end = END_N_SIZES,

        # Param
        **MPC_PARAM_DICT,
    ) for NN_VERSION in range(NUM_NNS) \
        for N_MPC, N_NN, N_DSET, TRAIN_DATASET_VERSION, TEST_DATASET_VERSION, USE_BEGIN,\
            DS_FEATURES, DS_OPT_NAME, N_HIDDEN, END_N_SIZES in COMBINED_NH_AMPC_OPTIONS 
]

### Acados options

In [6]:
# qp_solver
# ---------
#       'FULL_CONDENSING_QPOASES', 'FULL_CONDENSING_HPIPM', 'FULL_CONDENSING_DAQP',
#       'PARTIAL_CONDENSING_HPIPM', 'PARTIAL_CONDENSING_QPDUNES', 'PARTIAL_CONDENSING_OSQP'
# hessian_approx
# --------------
#       'GAUSS_NEWTON', 'EXACT'
# integrator_type
# ---------------
#       'ERK', 'IRK', 'DISCRETE', 'LIFTED_IRK', 'GNSF'
# nlp_solver_type
# ---------------
#       'SQP_RTI', 'SQP'
# regularize_method
# -----------------
#       'NO_REGULARIZE', 'MIRROR', 'PROJECT', 'PROJECT_REDUC_HESS', 'CONVEXIFY'
# hpipm_mode
# ----------
#       'BALANCE', 'SPEED_ABS', 'SPEED', 'ROBUST'
# collocation_type
# ----------------
#       'GAUSS_RADAU_IIA', 'GAUSS_LEGENDRE'
# globalization
# -------------
#       'FIXED_STEP', 'MERIT_BACKTRACKING'
# nlp_solver_tol_stat
# -------------------
#       Default: 1e-6
# nlp_solver_max_iter
# -------------------
#       Default: 100
# as_rti_iter
# -----------
#       Default: 1
# as_rti_level
# ------------
#       Default: 4
# 
#############################################
# use_iter_rti_impl
# -----------------
#       Default = False
# use_initial_guesses
# -----------------
#       Default = False
# max_rti_iters
# -----------------
#       Default = 10
# rti_tol
# -----------------
#       Default = 1e-4





SOLVER_OPTIONS = {
    f'ASRTID_FCH': dict(
        qp_solver='FULL_CONDENSING_HPIPM', 
        integrator_type='DISCRETE', 
        nlp_solver_type='SQP_RTI',
        as_rti_iter=3,
        as_rti_level=3,
        nlp_solver_tol_stat=1e-6,
        nlp_solver_max_iter=3,
    )
}

## Load Networks and generate NH-AMPC trajectory

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() and USE_CUDA else 'cpu') 
dtype = torch.float32
print(device)

cpu


In [ ]:
with tqdm_notebook(total=len(SOLVER_OPTIONS)*len(NH_AMPC_PARAMS), unit='MPC', desc='Get trajectories: ') as tqdm_handle:
    for nh_ampc_param in NH_AMPC_PARAMS:
        for solver_option in SOLVER_OPTIONS.items():
            name = f'{solver_option[0]} {nh_ampc_param.N_MPC}M_{nh_ampc_param.N_NN}N {nh_ampc_param.N_hidden}Nh_{nh_ampc_param.V_NN}v' \
                if nh_ampc_param.N_hidden_end is None else \
                f'{solver_option[0]} {nh_ampc_param.N_MPC}M_{nh_ampc_param.N_NN}N {nh_ampc_param.N_hidden}Nh_{nh_ampc_param.N_hidden_end}NhP_{nh_ampc_param.V_NN}v'
            tqdm_handle.set_description_str(f'Get trajectory of:\n{name}')
            
            results = generate_NH_AMPC_trajs(nh_ampc_param, solver_option, MPC_RESULTS_DIR, NNS_DIR, MPC_DATASETS_DIR, device, dtype)

            tqdm_handle.update(1)
            tqdm_handle.set_postfix_str(f'\n{name}:\nCost:{results.Cost:.2f}, Time:{np.mean(results.Time)*1e3:.2f}ms, ATime:{np.mean(results.Acados_Time)*1e3:.2f}ms')

Get trajectories:   0%|          | 0/400 [00:00<?, ?MPC/s]

Model loaded from file "/home/jovyan/work/acadosmpc/Results/Trained_Networks/NN_acados_8M_17N_70ND_5VD_30000Dp_RTI_PCHPIPM_DISCRETE_24Nhid_0v.ph".
Model hyperparameters:
Feature names: ['px_p8', 'theta_p8', 'v_p8', 'omega_p8']
Label names: [px_p9, theta_p9, ..., v_p25, omega_p25] (68 elements)
Activation function: tanh
Number of hidden layers: 3 with [24, 24, 24] neurons
Size of outer layers: 4 input neurons, 68 output neurons
NN evaluation:
NN: [4]->[68], 3 layer(s), [4, 24, 24, 24] neuron(s) per layer
R2-score: 0.9560
Relative error: 5.09% mean, 10.37% standard deviation
